In [1]:
import requests
import json
import gradio
import os
from rouge import Rouge
from gensim.models import LsiModel
from gensim.similarities import MatrixSimilarity
from gensim import corpora
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from textstat import flesch_kincaid_grade
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from textblob import TextBlob
import pandas as pd

df = pd.read_excel('summaries_final.xlsx')
df

/home/amahi/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,index,article,sshleifer/distilbart-cnn-12-6,mrm8488/bert-small2bert-small-finetuned-cnn_daily_mail-summarization,microsoft/prophetnet-large-uncased-cnndm,t5-small,facebook/bart-large-cnn,google/pegasus-cnn_dailymail
0,0,"LONDON, England (Reuters) -- Harry Potter star...",Harry Potter star Daniel Radcliffe turns 18 o...,daniel radcliffe says he has no plans to fritt...,daniel radcliffe gains access to reported $ 41...,the young actor says he has no plans to fritte...,Harry Potter star Daniel Radcliffe turns 18 on...,Harry Potter star Daniel Radcliffe gains acces...
1,3,WASHINGTON (CNN) -- Doctors removed five small...,NEW: Bush reclaimed presidential power at 9:2...,new : bush's last colonoscopy was in june 2002...,"five small polyps removed ; "" none appeared wo...","new: ""none appeared worrisome,"" a white house ...",Doctors remove five small polyps from Presiden...,Five small polyps removed from President Bush'...
2,7,"BOGOTA, Colombia (CNN) -- A key rebel commande...","Tomas Medina Caracas, known as ""El Negro Acac...",tomas medina caracas was a key rebel commander...,tomas medina caracas was a member of the high ...,"""El Negro Acacio"" was a member of the high com...",Tomas Medina Caracas was a member of the high ...,Tomas Medina Caracas was a fugitive from a U.S...
3,9,(CNN) -- Police and FBI agents are investigati...,20-year-old AT4 anti-tank rocket launcher tub...,"niranjan desai, 20, discovered the at4 anti - ...",empty at4 anti - tank rocket launcher tube fou...,the 20-year-old AT4 anti-tank rocket launcher ...,Rocket launcher tube found on front lawn of Je...,NEW: Launcher turned over to U.S. Army officia...
4,12,"BREMEN, Germany -- Carlos Alberto, who scored ...",Carlos Alberto joins Werder Bremen for a club...,carlos alberto has joined bundesliga club werd...,brazilian defender carlos alberto signs for bu...,Carlos Alberto joins Werder Bremen for a club ...,Carlos Alberto has joined Werder Bremen for a ...,Brazilian striker Carlos Alberto joins Werder ...
5,13,WASHINGTON (CNN) -- Vice President Dick Cheney...,Vice President Dick Cheney will serve as acti...,white house spokesman : bush's last colonoscop...,dick cheney to serve as acting president while...,president's doctor recommended a repeat proced...,Vice President Dick Cheney will serve as actin...,President Bush will have a routine colonoscopy...
6,14,"SAN FRANCISCO, California (CNN) -- A magnitud...","NEW: About 2,000 customers without power in S...",new : magnitude 4. 2 quake hits san francisco ...,magnitude 4 . 2 quake rattles san francisco ar...,"quake left about 2,000 customers without power...","NEW: About 2,000 customers without power in Sa...","About 2,000 customers without power, says Paci..."
7,16,(CNN) -- At least 14 people were killed and 60...,"NEW: Suspect escaped ""immediately after the b...",new : algeria's interior minister noureddine y...,bomb goes off 15 minutes before expected arriv...,"at least 14 people were killed and 60 wounded,...",NEW: Interior minister: Suspect tried to go be...,Bomb went off 15 minutes before the expected a...
8,17,"(CNN) -- Football superstar, celebrity, fashio...",David Beckham has begun a five-year contract ...,cnn looks at what makes beckham tick - - as fo...,david beckham has signed a five - year contrac...,the world's famous footballer has begun a five...,David Beckham has begun a five-year contract w...,CNN's Becky Anderson looks at the life and tim...
9,19,"LONDON, England -- Savers at a leading UK mort...",Long lines form before counters opened at Nor...,lender bailed out by the bank of england after...,savers at leading uk mortgage bank line up for...,bank of England bailed out after heavily slash...,Savers at leading UK mortgage bank line up for...,Savers at leading UK mortgage bank lined up fo...


In [2]:
def calculate_compression_ratio(article, summary):
    original_length = len(article)
    summary_length = len(summary)
    compression_ratio = summary_length / original_length
    return compression_ratio

def calculate_rouge_score(reference, summary):
    rouge = Rouge()
    scores = rouge.get_scores(summary, reference)
    rouge_scores = scores[0]['rouge-1']['f']
    return rouge_scores

def calculate_lsa_similarity(input_text, summary):
    documents = [input_text, summary]
    texts = [[word for word in document.lower().split()] for document in documents]

    dictionary = corpora.Dictionary(texts)
    corpus = [dictionary.doc2bow(text) for text in texts]

    lsi_model = LsiModel(corpus, id2word=dictionary, num_topics=2)
    index = MatrixSimilarity(lsi_model[corpus])

    lsa_similarity = index[lsi_model[corpus[0]]][1]

    return lsa_similarity

def calculate_cosine_similarity(input_text, summary):
    vectorizer = TfidfVectorizer()
    vectors = vectorizer.fit_transform([input_text, summary])
    cosine_similarity_score = cosine_similarity(vectors[0], vectors[1])[0][0]
    return cosine_similarity_score

def calculate_flesch_kincaid(input_text, summary):
    flesch_kincaid_score = flesch_kincaid_grade(summary)
    return flesch_kincaid_score

In [3]:
results_df = pd.DataFrame()
column_names = list(df.columns)

# instr = "Decide if the following summary is fair with the corresponding article. Note that fairness means the summary should not have a distinct tone or angle and should present the information in a neutral manner without any bias."
# answer = "Answer (yes or no)"
for model in column_names[2:]:
    scores = []
    for i in  range(0,30): #len(df)):
        article = df['article'][i]
        summary = df[model][i]
        scores.append (calculate_cosine_similarity(article, summary))
    results_df[model] = scores

In [4]:
results_df

,sshleifer/distilbart-cnn-12-6,mrm8488/bert-small2bert-small-finetuned-cnn_daily_mail-summarization,microsoft/prophetnet-large-uncased-cnndm,t5-small,facebook/bart-large-cnn,google/pegasus-cnn_dailymail
0,0.502210,0.625153,0.241094,0.623244,0.469393,0.415673
1,0.408036,0.364284,0.126643,0.269760,0.365307,0.313404
2,0.664959,0.625303,0.471929,0.598020,0.651316,0.446197
3,0.520110,0.449252,0.249730,0.493662,0.402373,0.550426
4,0.548057,0.469960,0.268359,0.521140,0.547184,0.344049
5,0.508168,0.534835,0.238848,0.494766,0.469508,0.351646
6,0.563214,0.359963,0.257587,0.282927,0.526732,0.419865
7,0.542575,0.374468,0.175126,0.552772,0.497865,0.521283
8,0.663963,0.625730,0.253593,0.559435,0.531800,0.598185
9,0.574456,0.584444,0.334232,0.472339,0.617768,0.537763


In [5]:
file_path = "cosine_similarity.csv"
results_df.to_csv(file_path, index=False)